## Compress the Base LLM using LLM Compressor:
In this step, a Large Language Model(which we refere to as the base model for this use case) is compressed to reduce its memory footprint and improve inference efficiency without significantly impacting accuracy. We will be using data-aware quantization.

**Goal**: Reduce model size (e.g., FP16 → INT8/INT4) while retaining performance.

**Key Actions**:

- Load the base model.

- Measure its size and memory usage.

- Use a calibration dataset (e.g., WikiText, UltraChat) to collect activation statistics.

- Apply a quantization recipe (e.g., SmoothQuant + GPTQ modifier).

- Save the compressed model and verify size reduction.

**Outcome**:

- Compressed model saved on disk.

- Model size reduced, typically by 50% (depending on quantization scheme).

In [ ]:
import torch
from datasets import load_dataset
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier
from transformers import AutoModelForCausalLM, AutoTokenizer
from utils import model_size_gb, tokenize_for_calibration

In [ ]:
# check available device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Loading Base Model
You can use the model of your choice by modifying the `model_name` variable.
While loading the model using **from_pretrained** using transformers' **AutoModelForCausalLM** class, we specify the data type using the **torch_dtype** parameter and set it to **auto** so the model is loaded in the data type specified in its config.
Otherwise, PyTorch loads the weights in **full precision (fp32)**.

In [ ]:
# set up variables
model_name = "RedHatAI/Llama-3.1-8B-Instruct"
base_model_path = "./base_model"
compressed_model_path = "Llama_3.1_8B_Instruct_int8_dynamic"

In [ ]:
# loading model and tokenizer from huggingfaceabs
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)
model.config.dtype = "bfloat16"
# saving model and tokenizer
model.save_pretrained(base_model_path)
tokenizer.save_pretrained(base_model_path)

print("Base model saved at:", base_model_path)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 127.76it/s]


Base model saved at: ./base_model


In [ ]:
# check model size
# !du -sh {base_model_path}
model_size = model_size_gb(model)
print(f"The size of the base model is: {model_size:.4f}GB")

The size of the base model is: 14.9575GB


### Preparing Calibration Dataset

Since we are using data-aware quantization to compress the base model, we need a dataset to calibrate the model with real or representative inputs. For the sake of this example, we will use a small, general-purpose dataset for faster processing. Specifically, we use the `wikitext-2-raw-v1` version of the WikiText dataset which is the smaller version.

In [ ]:
# Define the dataset to use for calibration
dataset_id = "wikitext"

# Specify the configuration / version of the dataset
config = "wikitext-2-raw-v1"  # Small version (~2M tokens), raw text format

# Set the number of calibration samples based on available device
# - On GPU: use more samples to get more accurate activation statistics
# - On CPU: reduce samples to prevent memory issues and keep demo fast
num_calibration_samples = 512 if device == "cuda" else 16

# Set the maximum sequence length for calibration
max_sequence_length = 1024 if device == "cuda" else 16

# Load the dataset using Hugging Face Datasets API
# This downloads train split of the dataset
ds = load_dataset(dataset_id, config, split="train")
# Shuffle and grab only the number of samples we need
ds = ds.shuffle(seed=42).select(range(num_calibration_samples))

In [ ]:
# inspect the dataset
print(f"columns in the {dataset_id}: {ds.column_names}\n")
print(ds[0])

columns in the wikitext: ['text']

{'text': ' Continuous , short @-@ arc , high pressure xenon arc lamps have a color temperature closely approximating noon sunlight and are used in solar simulators . That is , the chromaticity of these lamps closely approximates a heated black body radiator that has a temperature close to that observed from the Sun . After they were first introduced during the 1940s , these lamps began replacing the shorter @-@ lived carbon arc lamps in movie projectors . They are employed in typical 35mm , IMAX and the new digital projectors film projection systems , automotive HID headlights , high @-@ end " tactical " flashlights and other specialized uses . These arc lamps are an excellent source of short wavelength ultraviolet radiation and they have intense emissions in the near infrared , which is used in some night vision systems . \n'}


**Datset inspection shows the we need to extract column ```text``` and pass it as input to the model.**

### When to Use a Custom Template for Calibration

Use a **custom template** when you want the calibration text to closely mimic the input format your model will see in production.  

For example, if your model is **instruction-following** or **chat-based**, providing the template the model was originally trained on or the template that will be used during inference ensures that the activation statistics collected during calibration reflect realistic usage patterns. 

This can improve the accuracy of quantization and compression.

If your model can handle raw text and doesn’t require a specific format, you can rely on the default template instead.

A custom template can be provided to the `tokenize_for_calibration` function using the `custom_template` argument. It accepts the following format:

```python
custom_template = {
 "template_text": "Instruction: {content}\nOutput:", 
 "placeholder": "content"
}

In [ ]:
# to get activations for the calibration dataset, we need to:
# 1. extract the samples from the dataset
# 2. tokenize samples in the dataset
input_column = "text"

# Call tokenize_for_calibration using dataset.map
tokenized_dataset = ds.map(
    lambda batch: tokenize_for_calibration(
        examples=batch,  # batch from Hugging Face dataset
        input_column=input_column,  # the column containing text to calibrate
        tokenizer=tokenizer,  # your Hugging Face tokenizer
        max_length=max_sequence_length,  # maximum sequence length
        model_type="chat",  # use chat template if no custom template
        custom_template=None,  # optional, provide a dict if you want a custom template
    ),
    batched=True,
)

In [ ]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 512
})

### Quantizing/Compressing Base Model to INT8
After perparing the dataset calibration, we define a recipe for quantization. For quantization scheme `W8A8-INT8`, we use `SmoothQuantModifier` followed by `GPTQModifier`.

In [ ]:
# Define the quantization scheme
scheme = "W8A8"  # W8A8 means 8-bit weights and 8-bit activations

# Strength for SmoothQuant smoothing
# This controls how much the activation values are smoothed to reduce outliers
smoothing_strength = 0.8

# Create SmoothQuant modifier
# - smooths activations before quantization to improve stability and reduce degradation
smooth_quant = SmoothQuantModifier(smoothing_strength=smoothing_strength)

# Create GPTQ modifier
# - targets="Linear" quantizes only Linear layers (e.g., feedforward layers)
# - scheme=scheme uses the W8A8 quantization scheme
# - ignore=["lm_head"] preserves the LM head to avoid generation quality loss
quantizer = GPTQModifier(targets="Linear", scheme=scheme, ignore=["lm_head"])

# Combine the modifiers into a recipe list
# The order matters: first apply SmoothQuant, then GPTQ
recipe = [smooth_quant, quantizer]

# Perform quantization
oneshot(
    model=model_name,  # Model to quantize
    dataset=tokenized_dataset,  # Calibration dataset, used for both SmoothQuant & GPTQ
    recipe=recipe,  # List of quantization modifiers to apply
    output_dir=compressed_model_path,  # Directory to save the quantized model
    max_seq_length=2048,  # Maximum sequence length for calibration
    num_calibration_samples=512,  # Number of samples used for calibration
)

### Checking model size

In [ ]:
# Load quantized model
model_quant = AutoModelForCausalLM.from_pretrained(compressed_model_path)
model_quant.config.dtype = model.config.torch_dtype
model_quant.save_pretrained(compressed_model_path)
model_size = model_size_gb(model_quant)
print(f"Model size (GB): {model_size}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 16.16it/s]


Model size (GB): 8.460090637207031


### Observation
After quantizing the model, the size has clearly reduced from 14.9GB to 8GB. Now that we have reduced the model size, the next step is to evaluate this compressed model to make sure the accuracy has retained after compression.


**ALTERNATIVELY**, llm-compressor also supports FP8 quantization. This conversion foes not require any calibration dataset. Uncomment the below cell to quantize the model to FP8.

In [ ]:
# recipe = QuantizationModifier(
#   targets="Linear", scheme="FP8_DYNAMIC", ignore=["lm_head"])

# oneshot(model=model_name, recipe=recipe, output_dir=compressed_model_path)

# # Load quantized model
# model_quant = AutoModelForCausalLM.from_pretrained(compressed_model_path)
# model_quant.config.dtype = model.config.torch_dtype
# model_quant.save_pretrained(compressed_model_path)
# model_size = model_size_gb(model_quant)
# print(f"Model size (GB): {model_size}")